## Interact with NGSI-LD context broker usign the data model

Uses orion-ld broker

copy updated context to kubernetes using:

In [21]:
from ngsildclient import Entity, Client

In [22]:
host = "10.0.0.1"
client = Client(hostname=host, port=1026, port_temporal=8080)
ctx = "http://nginx:8000/context-ngsi.jsonld"

Connected to Context Broker at 10.0.0.1:1026 | tenant=N/A | vendor=Orion-LD | version=1.1.2

In [24]:
# Create a cow
import datetime
import numpy as np
start = datetime.datetime.fromisoformat("2023-01-01T00:00:00")
cow = Entity("AnimalDailyObserved", "cow4", ctx = ctx)
cow.prop("methaneyield", np.random.randint(30, 60), observedat=start)
cow.prop("net_energy_requirement", np.random.randint(700, 1000), observedat=start)
#cow.tprop("observedAt", start) # This gives errors from orion
#cow["observedAt"] = start.isoformat() + "Z" # This works
#cow.obs() This one works for some reason
cow.pprint()
#client.create(cow)

{
  "id": "urn:ngsi-ld:AnimalDailyObserved:cow4",
  "type": "AnimalDailyObserved",
  "@context": "http://nginx:8000/context-ngsi.jsonld",
  "methaneyield": {
    "type": "Property",
    "value": 55,
    "observedAt": "2023-01-01T00:00:00Z"
  },
  "net_energy_requirement": {
    "type": "Property",
    "value": 879,
    "observedAt": "2023-01-01T00:00:00Z"
  }
}

In [36]:
%%time
# Create and update the values
for i in range(10):
    cow = Entity("AnimalDailyObserved", f"ttcow{i}", ctx = ctx)
    cow.prop("methaneyield", np.random.randint(30, 60))
    cow.prop("net_energy_requirement", np.random.randint(700, 1000))
    #cow.tprop("observedAt", start) # This gives errors from orion
    cow["observedAt"] = start.isoformat() + "Z" # This works
    #cow.obs() This one works for some reason
    #cow.pprint()
    #client.create(cow)
    for t in range(5):
        tn = start + datetime.timedelta(days=t)
        cow.prop("methaneyield", np.random.randint(30, 60), observedat=tn)
        cow.prop("net_energy_requirement", np.random.randint(700, 1000), observedat=tn)
        #cow.prop("observedAt", start.isoformat())
        cow["observedAt"] =tn.isoformat() + "Z"
        #cow.pprint()
        client.update(cow)

CPU times: user 301 ms, sys: 15.3 ms, total: 316 ms
Wall time: 923 ms


In [29]:
#client.query("AnimalDailyObserved", ctx=ctx)

## Query time series

In [30]:
# See https://ngsi-ld-tutorials.readthedocs.io/en/latest/short-term-history.html

query using requests:

In [37]:
import requests
import datetime
ctx = "http://nginx:8000/context-ngsi.jsonld"

#Mintaka on mikrok8s
url = 'http://10.0.0.2:8080/temporal/entities/' 
headers = {
    'Link': f'<{ctx}>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"',
}
params = {
    'type': 'AnimalDailyObserved',
    'pageSize': 100,
    'lastN': 10,
    'timeproperty': 'observedAt',
    'timerel': 'before',
    'timeAt': f'{datetime.datetime.now().isoformat()}',
    'options' : 'temporalValues'
}

response = requests.get(url, headers=headers, params=params)
cowdata = response.json()
cowdata[1]

{'id': 'urn:ngsi-ld:AnimalDailyObserved:ttcow6',
 'type': 'AnimalDailyObserved',
 'methaneyield': {'type': 'Property',
  'values': [[41.0, '2023-01-05T00:00:00Z'],
   [32.0, '2023-01-05T00:00:00Z'],
   [31.0, '2023-01-04T00:00:00Z'],
   [32.0, '2023-01-04T00:00:00Z'],
   [36.0, '2023-01-03T00:00:00Z'],
   [46.0, '2023-01-03T00:00:00Z'],
   [32.0, '2023-01-02T00:00:00Z'],
   [30.0, '2023-01-02T00:00:00Z'],
   [52.0, '2023-01-01T00:00:00Z'],
   [49.0, '2023-01-01T00:00:00Z']]},
 'net_energy_requirement': {'type': 'Property',
  'values': [[868.0, '2023-01-05T00:00:00Z'],
   [963.0, '2023-01-05T00:00:00Z'],
   [832.0, '2023-01-04T00:00:00Z'],
   [745.0, '2023-01-04T00:00:00Z'],
   [885.0, '2023-01-03T00:00:00Z'],
   [927.0, '2023-01-03T00:00:00Z'],
   [979.0, '2023-01-02T00:00:00Z'],
   [988.0, '2023-01-02T00:00:00Z'],
   [966.0, '2023-01-01T00:00:00Z'],
   [925.0, '2023-01-01T00:00:00Z']]}}

In [32]:
%%timeit
response = requests.get(url, headers=headers, params=params)

355 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
while True:
    response = requests.get(url, headers=headers, params=params)